### MODULES
All the modules, constant, imports and libraries used

In [ ]:
import pandas as pd

FILE_INPUT_TSV = 'pois_all_info'
INPUT_IDS = 'ny_municipality_pois_id.csv'

### READS THE MUNICIPALITY FILE
Reads the file containing the id of all the elements inside the municipality of New York. <br>
The file is organized as it follows: 
<ul>
<li>one id per row, where each id is a POI in New York municipality</li>
</ul>
Returns:
<ul>
<li>The set of IDs</li>
</ul>

In [ ]:
def readIds(file:str)->set[str]:
    

### READ TSV FILE
Reads the tsv file containing all the info, the file is organized as it follows:
<ul>
<li>@id: a unique id for each point of interest.</li>
<li>@lat: latitude coordinate of the POI in decimal degrees.</li>
<li>@lon: longitude coordinate of the POI in decimal degrees.</li>
<li>amenity name: if the POI category is amenity its type is reported in this field.</li>
<li>shop: if the POI category is shop its type is reported in this field.
<li>public_transport: if the POI category is public transport its type is reported in this field.</li>
<li>highway: if the POI category is highway its type is reported in this field.</li>
</ul>

Returns a dataFrame

## MAIN FUNCTION
This is the main function of the file that will control the flow of the program
